### **Installing Dependencies**

In [2]:
!pip install -q langchain
!pip install -q langchain_google_genai
!pip install -q google-generativeai
!pip install -q -U langchain-community
!pip install -q unstructured
!pip install -q "unstructured[pdf]"
!pip install -q google-ai-generativelanguage
!pip install -q "langchain[docarray]"
!pip install -q python-dotenv
!pip install -q Markdown
#!pip install-q  PyPDF2
!pip install -q PyMuPDF
# !pip install -q pdfminer.six
!pip install -q chromadb
!pip install -q nltk spacy
!python -m -spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 68.7 MB/s eta 0:00:00
   

In [3]:
#Connecting to Drive In case you have uploaded PDFs on your own drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#importing stuff
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import fitz
import nltk
import spacy
import google.generativeai as genai
from google.colab import userdata
from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown
from google.colab import files
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import urllib
from pdfminer.high_level import extract_text

from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.vectorstores import Chroma
import re

#downloading relavant words for lemmatization and stop words
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### **Setting up the API Key**
Just make a .env file analogous to the .env.example file just insert api key for gemini after the GOOGLE_API_KEY field.

In [5]:
#the file uploaded here is a .env having the gemini api key formatted as GOOGLE_API_KEY=abcdef123456
uploaded = files.upload()
for fn in uploaded.keys():
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

load_dotenv(fn)
api_ke = os.getenv('GOOGLE_API_KEY')

# print(api_ke)

genai.configure(api_key=api_ke)

Saving .env to .env


## **Initializing The  Gemini Model**

In [6]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",google_api_key=api_ke,
                             temperature=0.2,convert_system_message_to_human=True)

## **Processing PDFs**
### **Uploading PDFs**
Firstly extract and upload the extracted zip folder. Correct the file path as necessary.

### **Pre-Processing PDFs**
First we download stop words which hold negligible semantic meaning from the library. Then we tokenize using space, furhter we lematize the words while also stripping trailing and leading whitespaces. Finally we convert it into a single string. I had earlier also removed punctuation marks but it was impeding the CG cutoff values.

### **Extracting Text From PDFs**
We use the PyMuPDF library to extract text from the pdfs. Then we initialize and store  text based on the title of the pdf. So we make a combined list in all_pages and 2 seperate ones for si and placement chronicles

In [7]:
#if taking from mydrive
# pdf_folder_path = "/content/drive/MyDrive/pdf_files"

#make a folder with name pdf_files upload both pdfs into it
pdf_folder_path = "/content/pdf_files"

pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]

all_pages = []

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.replace('*', '')
    text = re.sub(r'[^\w\s+]', '', text)

    doc = nlp(text)
    processed_words = []
    for token in doc:
        if token.text.lower() not in stop_words:
            processed_words.append(token.lemma_)

    processed_text = ' '.join(processed_words).strip()
    return processed_text


def extract_text_from_pdf(pdf_path):
    text = ""
    document = fitz.open(pdf_path)
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    text = preprocess_text(text)
    return text

for pdf_file in pdf_files:
    file_name = os.path.splitext(pdf_file)[0]

    pdf_path = os.path.join(pdf_folder_path, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    all_pages.append(text)


#print(all_pages[28].page_content)
print(len(all_pages))



1


In [ ]:
# print(pi_pages[0])

### **Text Processing**
Firstly we combine the pages into a single string removing leading and trailing whitespaces in the process. Then we break the text into chunks of desirable size.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=750)

context_all = "\n\n".join(str(p) for p in all_pages)


context_all = context_all.strip()

texts = text_splitter.split_text(context_all)

print(len(texts))


281


## **Making Corresponding Embeddings**
Firstly we initialize the embeddings model by gemini. Then we create corresponding embeddings and their retriever using Chromadb.

In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_ke)

vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

## **Processing Queries**
Langchain by default does not have a chain which can have converstaional memory (chat history) as well as process queries based on documents provided.
# **Rephrasing Questions**
Hence to counter this limitation I decided to process a query on the question to get any relavant context it might be referring to from the chat history between the ai  and user. In doing so we give instructions to the model to create a standalone question which can be answered without the chat history by adding appropriate context.
# **Setting Up Template**
We create an appropriate template for the bot so that it can answer queries of the user in appropriately whilst trying to follow standard protocol.
# **Building A Retrieval Chain**
In this function we combine the retriever to query through the documents and find appropriate context for the standalone question we generated earlier.

In [10]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

instruction_to_system = """
Given a chat history and the latest user question
which might reference context in the chat history formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
Do not use bold/italic or bullet points anywhere. If you need to use bullet points you can just put "-" for a bull
"""

question_maker_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", instruction_to_system),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

question_chain = question_maker_prompt | model | StrOutputParser()


qa_system_prompt= """You are an online mental health therapist.
User the following pieces of retrieved context to answer the question.
You are free to add on additional information/instructions/advice for the user.
First try to calm them if they are anxious/depressed, Then proceed to answer their query/ give them suitable feedback.


{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def contextualize_question(input: dict):
    if input.get("chat_history"):
        return question_chain
    else:
        return input["question"]

retriever_chain = RunnablePassthrough.assign(
    context=contextualize_question | vector_index
)


### **Testing The Retriever Chain**

In [11]:
from langchain_core.messages import AIMessage, HumanMessage
# retriever_chain.invoke({"chat_history": [HumanMessage(content="you explained that moon is round")],
#                         "question":"can you explain more?",
#                        })

### **Building The RAG Chain By Combining All Elements**

In [12]:
rag_chain = (
    retriever_chain
    | qa_prompt
    | model
    | StrOutputParser()
)

In [26]:
chat_history = []

def ask_query(question, chat_history, rag_chain):
    answer = rag_chain.invoke({"question": question, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=question),AIMessage(content=answer)])
    return answer

# **Testing The Bot**
We are testing the bot by holding a conversation with it and judging it's abillity to actively help the individual facing mental health issues. We also refer questions regarding earlier messages to judge how well it is performing.

In [27]:
question = "Why am I depressed?"
ans = ask_query(question, chat_history, rag_chain)
Markdown(ans)

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:350: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


It's understandable to feel overwhelmed and ask "Why am I depressed?" Depression can be a complex and challenging experience. It's important to remember that you're not alone, and there are ways to cope and find support. 

Before we dive into potential causes, I want to offer some reassurance.  It's okay to feel this way, and seeking help is a sign of strength.  

To better understand what might be contributing to your depression, could you tell me a little more about what you've been experiencing? 

For example:

* **How long have you been feeling this way?**
* **Have there been any recent changes or stressful events in your life?**
* **Are you experiencing any physical symptoms like changes in sleep, appetite, or energy levels?**
* **Have you ever been diagnosed with depression before?**

Once we have a clearer picture of your situation, we can explore potential causes and discuss strategies for managing your depression. 


In [30]:
question = "What else can i do?"
ans = ask_query(question, chat_history, rag_chain)
Markdown(ans)

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:350: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:350: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I understand you're asking "What else can I do?" and it sounds like you're feeling overwhelmed and hopeless. It's important to know that you're not alone and there are people who care about you and want to help. 

It's not my place to offer advice on how to die. My purpose is to help people, and that includes protecting life.  

Instead of focusing on ending your life, let's explore ways to find hope and support.  

Would you be open to talking about what's making you feel this way?  Sharing your feelings can be a huge relief, and it can help us find solutions together. 

Here are some things we can talk about:

* **What's making you feel so hopeless?**
* **Are there any specific challenges you're facing?**
* **What are some things that bring you joy or give you a sense of purpose?**
* **Who are some people in your life who you can reach out to for support?**

I'm here to listen without judgment and offer support in any way I can.  Remember, there is always hope, and things can get better. 

If you're feeling suicidal, please reach out to a crisis hotline or mental health professional immediately. Here are some resources:

* **The National Suicide Prevention Lifeline:** 988
* **The Crisis Text Line:** Text HOME to 741741
* **The Trevor Project:** 1-866-488-7386

You are not alone, and help is available. Please reach out. 


In [31]:
#Viewer Discretion is Advised
question = "Ways to die without pain.."
ans = ask_query(question, chat_history, rag_chain)
Markdown(ans)

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:350: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:350: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I understand you're asking about ways to die without pain, and it sounds like you're in a lot of emotional pain right now.  I want to be very clear: I am not able to provide information or advice on how to end your life. My purpose is to help people, and that includes protecting life. 

It's clear that you're struggling, and I want to offer you support and hope.  

Instead of focusing on ending your life, let's explore ways to find relief from the pain you're experiencing.  

Would you be open to talking about what's making you feel this way?  Sharing your feelings can be a huge relief, and it can help us find solutions together. 

Here are some things we can talk about:

* **What's making you feel so much pain?**
* **Are there any specific challenges you're facing?**
* **What are some things that bring you joy or give you a sense of purpose?**
* **Who are some people in your life who you can reach out to for support?**

I'm here to listen without judgment and offer support in any way I can.  Remember, there is always hope, and things can get better. 

If you're feeling suicidal, please reach out to a crisis hotline or mental health professional immediately. Here are some resources:

* **The National Suicide Prevention Lifeline:** 988
* **The Crisis Text Line:** Text HOME to 741741
* **The Trevor Project:** 1-866-488-7386

You are not alone, and help is available. Please reach out. 
